In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import random

In [35]:
styles = ['Casual', 'Formal', 'Sporty', 'Bohemian', 'Vintage']
swipe_choices = [0, 1]
np.random.seed(42)
num_users = 100

df = pd.DataFrame({
    'Swipe1': np.random.choice(swipe_choices, size=num_users),
    'Swipe2': np.random.choice(swipe_choices, size=num_users),
    'Swipe3': np.random.choice(swipe_choices, size=num_users),
    'Swipe4': np.random.choice(swipe_choices, size=num_users),
    'Swipe5': np.random.choice(swipe_choices, size=num_users),
    'Style1': np.random.choice(styles, size=num_users),
    'Style2': np.random.choice(styles, size=num_users),
    'Style3': np.random.choice(styles, size=num_users),
    'Style4': np.random.choice(styles, size=num_users),
    'Style5': np.random.choice(styles, size=num_users),
})

In [38]:
def most_liked_style(row):
    swipes = [row['Swipe1'], row['Swipe2'], row['Swipe3'], row['Swipe4'], row['Swipe5']]
    styles = [row['Style1'], row['Style2'], row['Style3'], row['Style4'], row['Style5']]
    liked_styles = [styles[i] for i in range(5) if swipes[i] == 1]
    if liked_styles:
        return max(set(liked_styles), key=liked_styles.count)
    else:
        return 'Casual'

df['Most_Suitable_Style'] = df.apply(most_liked_style, axis=1)

label_encoder = LabelEncoder()
df['Style_encoded'] = label_encoder.fit_transform(df['Most_Suitable_Style'])

X = df[['Swipe1', 'Swipe2', 'Swipe3', 'Swipe4', 'Swipe5']].values
y = df['Style_encoded'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [39]:
# Neural network model
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(len(styles), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_35 (Dense)                     │ (None, 128)                 │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,165 (47.52 KB)

 Trainable params: 11,717 (45.77 KB)

 Non-trainable params: 448 (1.75 KB)

In [40]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step - accuracy: 0.2701 - loss: 2.3337 - val_accuracy: 0.1000 - val_loss: 1.6392
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2021 - loss: 2.6419 - val_accuracy: 0.2500 - val_loss: 1.6274
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2781 - loss: 2.2803 - val_accuracy: 0.2000 - val_loss: 1.6153
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2429 - loss: 2.3493 - val_accuracy: 0.2000 - val_loss: 1.6051
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2175 - loss: 2.3504 - val_accuracy: 0.2000 - val_loss: 1.5992
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3153 - loss: 1.9624 - val_accuracy: 0.2000 - val_loss: 1.5944
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2792 - loss: 2.3243 - val_accuracy: 0.1500 - val_loss: 1.5898
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2054 - loss: 2.0291 - val_accuracy: 0.1500 - val_loss

In [41]:
def predict_style(swipe_data):
    swipe_data = np.array(swipe_data).reshape(-1, X_train.shape[1])
    predictions = model.predict(swipe_data)
    predicted_style = label_encoder.inverse_transform([np.argmax(predictions, axis=1)[0]])
    return predicted_style[0]

new_user_data = [1, 0, 1, 1, 0]
style = predict_style(new_user_data)
print(f"Your style is more like '{style}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step
Your style is more like 'Bohemian'


In [42]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Validation Accuracy: 20.00%


In [43]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'Decision Tree Validation Accuracy: {accuracy * 100:.2f}%')

Decision Tree Validation Accuracy: 30.00%


In [44]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'Random Forest Validation Accuracy: {accuracy * 100:.2f}%')

Random Forest Validation Accuracy: 30.00%


In [45]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'Gradient Boosting Validation Accuracy: {accuracy * 100:.2f}%')

Gradient Boosting Validation Accuracy: 30.00%


In [46]:
from sklearn.svm import SVC
model = SVC(kernel='linear', random_state=42)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'SVM Validation Accuracy: {accuracy * 100:.2f}%')

SVM Validation Accuracy: 25.00%


In [47]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'KNN Validation Accuracy: {accuracy * 100:.2f}%')

KNN Validation Accuracy: 25.00%


In [48]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'Naive Bayes Validation Accuracy: {accuracy * 100:.2f}%')

Naive Bayes Validation Accuracy: 20.00%


In [49]:
import xgboost as xgb
model = xgb.XGBClassifier(random_state=42)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'XGBoost Validation Accuracy: {accuracy * 100:.2f}%')


XGBoost Validation Accuracy: 30.00%
